In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets, transforms
import torch.nn.functional
import random
import datetime, os
from tqdm import trange, tqdm

In [2]:
%run Dataset_Loading.ipynb
%run Model_Loading.ipynb
%run S_Rprop.ipynb
%run S_Rprop_V2.ipynb

tensor([-1.1369e-16, -2.2027e-16, -2.5224e-16, -2.1316e-17,  4.3077e-17,
         4.0901e-16,  9.9476e-17, -6.0396e-17], dtype=torch.float64) tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       dtype=torch.float64)
tensor([0.3540, 0.6460], dtype=torch.float64) tensor([0.4787, 0.4787], dtype=torch.float64)
torch.Size([500, 8])


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
import wandb
wandb.login()

wandb: Currently logged in as: andersoren (sorenandersenbachelorthesis). Use `wandb login --relogin` to force relogin


True

In [5]:
dataset = "MNIST"
network = "Standard_no_D"
optim_mthd = "S-RpropV2"
minibatches = True  
n_epochs = 5
learning_rate = 5e-3
batch_size_train = [600]   #300 still needs 3 runs + delete latest run 3 in WandB
#lr_batch_sizes = [50, 100, 200, 500, 1000, 2500, 5000, 10000, 20000, 30000, 60000]
lr_batch_sizes = [3000, 6000, 15000]
eta_m, eta_p = 0.5, 1.2   
min_lr, max_lr = 1e-10, 1e-2

In [6]:
random_seed = 3
torch.manual_seed(random_seed)

In [7]:
train_losses = []
train_counter = []
lr_updates = []

In [8]:
plots = []
for lr_batch_size in lr_batch_sizes:
#for minibatch_size in batch_size_train:
    minibatch_size = batch_size_train[0]
    lr_debug = 0
    if optim_mthd == "Rprop":
        lr_batch_size = minibatch_size
    train_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])), batch_size=minibatch_size, shuffle=True)
    for j in range(5):
        training_loss = []
        print(minibatch_size,lr_batch_size, j)
        if network == "Standard":
            model = Big_Net()
        elif network == "Medium":
            model = Medium_Net()
        elif network == "Standard_no_D":
            model = Big_Net_no_D()
        elif network == "Deep":
            model = Deep_Net()
        model.to(device)
        if optim_mthd == "SGD":
            optimizer = optim.SGD(model.parameters(), lr=learning_rate)
        elif optim_mthd == "Rprop":
            optimizer = optim.Rprop(model.parameters(), lr=learning_rate, step_sizes=(min_lr, max_lr))
        elif optim_mthd == "S-RpropV0)":
            optimizer = stochastic_adaptive(model.parameters(), lr=learning_rate, etas=(eta_m, eta_p), step_sizes=(min_lr, max_lr))
        elif optim_mthd == "S-Rprop":
            optimizer = SRPROP(model.parameters(), M=minibatch_size, L=lr_batch_size, lr=learning_rate, etas=(eta_m, eta_p), step_sizes=(min_lr, max_lr))
        run = wandb.init(project="MNIST",
                                     name=f"Run {j+1+3}",
                                     config={"epochs": n_epochs,
                                             "optim_mthd": optim_mthd,
                                             "learning_rate": learning_rate,
                                             "dataset": dataset,
                                             #"h_Layers": 3,
                                             #"nodes_p_layer": 32,
                                             "minibatches": minibatches,
                                             "minibatch_size": minibatch_size if minibatches else 60000,
                                             #"Dropout_layers": False,
                                             "lr_batch_size": lr_batch_size})
                                             #"eta-, eta+": (eta_m, eta_p),
                                             #"min_lr, max_lr": (min_lr, max_lr),
                                             #"Re-run": "V2"})
         #                                    "Final": True})
        for epoch in range(1, n_epochs + 1):
            model.train()
            wandb_loss = 0
            #debug_losses = []
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = data.to(device), target.to(device)
                optimizer.zero_grad()
                output = model(data)
                loss = F.nll_loss(output, target)
                loss.backward()
                optimizer.step()
                wandb_loss += loss
                if ((batch_idx+1)*len(data))%3000==0:
                    if minibatch_size<3000:
                        wandb_loss /= (3000/len(data))
                    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, (batch_idx+1) * len(data),
                    len(train_loader.dataset), 100. * batch_idx / len(train_loader), wandb_loss.item()))
                    wandb.log({"Loss over latest 3000 patterns": wandb_loss})
                    training_loss.append(wandb_loss)
                    wandb_loss = 0
            train_loader = Shuffle_MNIST_Data(train_loader, random_seed)
        wandb.finish()
        plots.append(training_loss)

600 3000 0


C:\Users\soren\AppData\Local\Temp\ipykernel_12244\1195817258.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [3000/60000 (4%)]	Loss: 2.077485


C:\Users\soren\AppData\Local\Temp\ipykernel_12244\1195817258.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [6000/60000 (9%)]	Loss: 1.602040
Train Epoch: 1 [9000/60000 (14%)]	Loss: 1.186208
Train Epoch: 1 [12000/60000 (19%)]	Loss: 0.852628
Train Epoch: 1 [15000/60000 (24%)]	Loss: 0.596999
Train Epoch: 1 [18000/60000 (29%)]	Loss: 0.530070
Train Epoch: 1 [21000/60000 (34%)]	Loss: 0.379118
Train Epoch: 1 [24000/60000 (39%)]	Loss: 0.342087
Train Epoch: 1 [27000/60000 (44%)]	Loss: 0.261661
Train Epoch: 1 [30000/60000 (49%)]	Loss: 0.220252
Train Epoch: 1 [33000/60000 (54%)]	Loss: 0.211076
Train Epoch: 1 [36000/60000 (59%)]	Loss: 0.187439
Train Epoch: 1 [39000/60000 (64%)]	Loss: 0.144955
Train Epoch: 1 [42000/60000 (69%)]	Loss: 0.147502
Train Epoch: 1 [45000/60000 (74%)]	Loss: 0.134711
Train Epoch: 1 [48000/60000 (79%)]	Loss: 0.113394
Train Epoch: 1 [51000/60000 (84%)]	Loss: 0.135987
Train Epoch: 1 [54000/60000 (89%)]	Loss: 0.110224
Train Epoch: 1 [57000/60000 (94%)]	Loss: 0.099448
Train Epoch: 1 [60000/60000 (99%)]	Loss: 0.108190
Train Epoch: 2 [3000/60000 (4%)]	Loss: 0.091334
Train

...

Train Epoch: 1 [3000/60000 (4%)]	Loss: 2.170401
Train Epoch: 1 [6000/60000 (9%)]	Loss: 1.633800
Train Epoch: 1 [9000/60000 (14%)]	Loss: 1.215945
Train Epoch: 1 [12000/60000 (19%)]	Loss: 0.926262
Train Epoch: 1 [15000/60000 (24%)]	Loss: 0.835298
Train Epoch: 1 [18000/60000 (29%)]	Loss: 0.674795
Train Epoch: 1 [21000/60000 (34%)]	Loss: 0.561561
Train Epoch: 1 [24000/60000 (39%)]	Loss: 0.532436
Train Epoch: 1 [27000/60000 (44%)]	Loss: 0.429350
Train Epoch: 1 [30000/60000 (49%)]	Loss: 0.395631
Train Epoch: 1 [33000/60000 (54%)]	Loss: 0.415930
Train Epoch: 1 [36000/60000 (59%)]	Loss: 0.357044
Train Epoch: 1 [39000/60000 (64%)]	Loss: 0.338559
Train Epoch: 1 [42000/60000 (69%)]	Loss: 0.308918
Train Epoch: 1 [45000/60000 (74%)]	Loss: 0.299522
Train Epoch: 1 [48000/60000 (79%)]	Loss: 0.236399
Train Epoch: 1 [51000/60000 (84%)]	Loss: 0.203934
Train Epoch: 1 [54000/60000 (89%)]	Loss: 0.199642
Train Epoch: 1 [57000/60000 (94%)]	Loss: 0.219628
Train Epoch: 1 [60000/60000 (99%)]	Loss: 0.203392
Train

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001E560294EB0>> (for post_run_cell):


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "C:\Users\soren\.conda\envs\tf\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\soren\.conda\envs\tf\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\soren\.conda\envs\tf\lib\site-packages\wandb\sdk\wandb_run.py", line 286, in check_stop_status
    self._loop_check_status(
  File "C:\Users\soren\.conda\envs\tf\lib\site-packages\wandb\sdk\wandb_run.py", line 224, in _loop_check_status
    local_handle = request()
  File "C:\Users\soren\.conda\envs\tf\lib\site-packages\wandb\sdk\interface\interface.py", line 828, in deliver_stop_status
    return self._deliver_stop_status(status)
  File "C:\Users\soren\.conda\envs\tf\lib\site-packages\wandb\sdk\interface\interface_shared.py", line 494, in _deliver_stop_status
    return self._deliver_record(record)
  File "C:\Users\soren\.conda\envs\tf\lib\site-packages\wandb\sdk\interface\interface_